# 6  Data Loading, Storage, and File Formats

Reading data and making it accessible (often called *data loading*) is a necessary first step for using most of the tools in this book. The term *parsing* is also sometimes used to describe loading text data and interpreting it as tables and different data types. I’m going to focus on data input and output using pandas, though there are numerous tools in other libraries to help with reading and writing data in various formats.

Input and output typically fall into a few main categories: reading text files and other more efficient on-disk formats, loading data from databases, and interacting with network sources like web APIs.

## 6.1 Reading and Writing Data in Text Format

pandas features a number of functions for reading tabular data as a DataFrame object. **[Table 6.1](https://wesmckinney.com/book/accessing-data#tbl-table_parsing_functions)** summarizes some of them; `pandas.read_csv` is one of the most frequently used in this book.



I’ll give an overview of the mechanics of these functions, which are meant to convert text data into a DataFrame. The optional arguments for these functions may fall into a few categories:

* **Indexing**
Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, arguments you provide, or not at all.

* **Type inference and data conversion**
Includes the user-defined value conversions and custom list of missing value markers.

* **Date and time parsing**
Includes a combining capability, including combining date and time information spread over multiple columns into a single column in the result.

* **Iterating**
Support for iterating over chunks of very large files.

* **Unclean data issues**
Includes skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.


Because of how messy data in the real world can be, some of the data loading functions (especially `pandas.read_csv`) have accumulated a long list of optional arguments over time. It's normal to feel overwhelmed by the number of different parameters (`pandas.read_csv` has around 50). The online pandas documentation has many examples about how each of these works, so if you're struggling to read a particular file, there might be a similar enough example to help you find the right parameters.

Some of these functions perform `type inference` because the column data types are not part of the data format. That means you don’t necessarily have to specify which columns are numeric, integer, Boolean, or string. Other data formats, like HDF5, ORC, and Parquet, have the data type information embedded in the format.

Handling dates and other custom types can require extra effort.

Let’s start with a small comma-separated values (CSV) text file:

In [1]:
!cat /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

**Note:** Here I used the Unix `cat` shell command to print the raw contents of the file to the screen. If you’re on Windows, you can use `type` instead of `cat` to achieve the same effect within a Windows terminal (or command line).

Since this is comma-delimited, we can then use `pandas.read_csv` to read it into a DataFrame:



In [2]:
import pandas as pd

df = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Consider this file:



In [3]:
!cat /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex2.csv


1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yourself:



In [4]:
pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the `message` column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named `"message"` using the `index_col` argument:



In [6]:
names = ["a", "b", "c", "d", "message"]

pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


If you want to form a hierarchical index (discussed in Ch 8.1: Hierarchical Indexing) from multiple columns, pass a list of column numbers or names:


In [7]:
!cat /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/csv_mindex.csv


key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [8]:
parsed = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/csv_mindex.csv", index_col=["key1", "key2"])
                                                                                                               
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:



In [9]:
!cat /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


While you could do some munging by hand, the fields here are separated by a variable amount of whitespace. In these cases, you can pass a *regular expression* as a delimiter for `pandas.read_csv`. This can be expressed by the regular expression `\s+`, so we have then:



In [10]:
result = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex3.txt", sep="\s+")

result

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/var/folders/fn/3w5085ns7nv4v62cfvj7m9sm0000gp/T/ipykernel_75892/3174779491.py:1: SyntaxWarning: invalid escape sequence '\s'
  result = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex3.txt", sep="\s+")


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows, `pandas.read_csv` infers that the first column should be the DataFrame’s index in this special case.

The file parsing functions have many additional arguments to help you handle the wide variety of exception file formats that occur (see a partial listing in Table 6.2). For example, you can skip the first, third, and fourth rows of a file with `skiprows`:

In [11]:
!cat /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex4.csv



# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [12]:
pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file reading process. Missing data is usually either not present (empty string) or marked by some *sentinel* (placeholder) value. By default, pandas uses a set of commonly occurring sentinels, such as `NA` and `NULL`:



In [13]:
!cat /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [14]:
result = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Recall that pandas outputs missing values as `NaN` so we have two null or missing values in `result`:



In [15]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The `na_values` option accepts a sequence of strings to add to the default list of strings recognized as missing:



In [16]:
result = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex5.csv", na_values=["NULL"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


`pandas.read_csv` has a list of many default NA value representations, but these defaults can be disabled with the `keep_default_na` option:


In [17]:
result2 = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex5.csv", keep_default_na=False)

result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [18]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [19]:
result3 = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex5.csv", keep_default_na=False, na_values=["NA"])

result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


Different NA sentinels can be specified for each column in a dictionary:


In [20]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}

pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex5.csv", na_values=sentinels, keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


**[Table 6.2](https://wesmckinney.com/book/accessing-data#tbl-table_read_csv_function)** lists some frequently used options in `pandas.read_csv`.



### Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to correctly process a large file, you may want to read only a small piece of a file or iterate through smaller chunks of the file.

Before we look at a large file, we make the pandas display settings more compact:

In [21]:
pd.options.display.max_rows = 10

In [22]:
result = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


The elipsis marks `...` indicate that rows in the middle of the DataFrame have been omitted.

If you want to read only a small number of rows (avoiding reading the entire file), specify that with `nrows`:

In [23]:
pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read a file in pieces, specify a `chunksize` as a number of rows:



In [24]:
chunker = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex6.csv", chunksize=1000)

type(chunker)

pandas.io.parsers.readers.TextFileReader

The `TextFileReader` object returned by `pandas.read_csv` allows you to iterate over the parts of the file according to the `chunksize`. For example, we can iterate over `ex6.csv`, aggregating the value counts in the `"key"` column, like so:



In [25]:
chunker = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [26]:
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

`TextFileReader` is also equipped with a `get_chunk` method that enables you to read pieces of an arbitrary size.


### Writing Data to Text Format

Data can also be exported to a delimited format. Let’s consider one of the CSV files read before:

In [27]:
data = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex5.csv")

data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame’s `to_csv` method, we can write the data out to a comma-separated file:



In [28]:
data.to_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/out.csv")

In [29]:
!cat /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Other delimiters can be used, of course (writing to `sys.stdout` so it prints the text result to the console rather than a file):



In [30]:
import sys

data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:


In [31]:
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


With no other options specified, both the row and column labels are written. Both of these can be disabled:



In [32]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


You can also write only a subset of the columns, and in an order of your choosing:

In [33]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### Working with Other Delimited Formats

It's possible to load most forms of tabular data from disk using functions like `pandas.read_csv`. In some cases, however, some manual processing may be necessary. It’s not uncommon to receive a file with one or more malformed lines that trip up `pandas.read_csv`. To illustrate the basic tools, consider a small CSV file:



In [34]:
!cat /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


For any file with a single-character delimiter, you can use Python’s built-in `csv` module. To use it, pass any open file or file-like object to `csv.reader`:



In [35]:
import csv

f = open("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex7.csv")

reader = csv.reader(f)

Iterating through the reader like a file yields lists of values with any quote characters removed:

In [36]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


From there, it’s up to you to do the wrangling necessary to put the data in the form that you need. Let's take this step by step. First, we read the file into a list of lines:



In [37]:
with open("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex7.csv") as f:
    lines = list(csv.reader(f))

Then we split the lines into the header line and the data lines:


In [38]:
header, values = lines[0], lines[1:]

Then we can create a dictionary of data columns using a dictionary comprehension and the expression `zip(*values)` (beware that this will use a lot of memory on large files), which transposes rows to columns:



In [39]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV files come in many different flavors. To define a new format with a different delimiter, string quoting convention, or line terminator, we could define a simple subclass of `csv.Dialect`:



In [40]:
class my_dialect(csv.Dialect):
    lineterminator = "\n"
    delimiter = ";"
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

reader = csv.reader(f, delimiter="|")

ValueError: I/O operation on closed file.

The possible options (attributes of `csv.Dialect`) and what they do can be found in **[Table 6.3](https://wesmckinney.com/book/accessing-data#tbl-table_csv_dialect)**.

To *write* delimited files manually, you can use `csv.writer`. It accepts an open, writable file object and the same dialect and format options as `csv.reader`:



In [ ]:
with open("mydata.csv", "w") as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(("one", "two", "three"))
    writer.writerow(("1", "2", "3"))
    writer.writerow(("4", "5", "6"))
    writer.writerow(("7", "8", "9"))

### JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. Here is an example:



In [ ]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

JSON is very nearly valid Python code with the exception of its null value `null` and some other nuances (such as disallowing trailing commas at the end of lists). The basic types are objects (dictionaries), arrays (lists), strings, numbers, Booleans, and nulls. All of the keys in an object must be strings. There are several Python libraries for reading and writing JSON data. I’ll use `json` here, as it is built into the Python standard library. To convert a JSON string to Python form, use `json.loads`:



In [ ]:
import json

result = json.loads(obj)

result

`json.dumps`, on the other hand, converts a Python object back to JSON:



In [ ]:
asjson = json.dumps(result)

asjson

How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, you can pass a list of dictionaries (which were previously JSON objects) to the DataFrame constructor and select a subset of the data fields:



In [ ]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])

siblings

The `pandas.read_json` can automatically convert JSON datasets in specific arrangements into a Series or DataFrame. For example:



In [ ]:
!cat /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/example.json

The default options for `pandas.read_json` assume that each object in the JSON array is a row in the table:



In [ ]:
data = pd.read_json("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/example.json")
data

If you need to export data from pandas to JSON, one way is to use the `to_json` methods on Series and DataFrame:



In [ ]:
data.to_json(sys.stdout)

In [ ]:
data.to_json(sys.stdout, orient="records")

### XML and HTML: Web Scraping

Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is comparatively much faster in general, the other libraries can better handle malformed HTML or XML files.

pandas has a built-in function, `pandas.read_html`, which uses all of these libraries to automatically parse tables out of HTML files as DataFrame objects. To show how this works, I downloaded an HTML file (used in the pandas documentation) from the US FDIC showing bank failures.1 First, you must install some additional libraries used by `read_html`:

`conda install lxml beautifulsoup4 html5lib`

If you are not using conda, `pip install lxml` should also work.

The `pandas.read_html` function has a number of options, but by default it searches for and attempts to parse all tabular data contained within `<table> `tags. The result is a list of DataFrame objects:



In [ ]:
pip install lxml

In [ ]:
tables = pd.read_html("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/fdic_failed_bank_list.html")

len(tables)

In [ ]:
failures = tables[0]

failures.head()

Because failures has many columns, pandas inserts a line break character `\`.

As you will learn in later chapters, from here we could proceed to do some data cleaning and analysis, like computing the number of bank failures by year:

In [ ]:
close_timestamps = pd.to_datetime(failures["Closing Date"])

close_timestamps.dt.year.value_counts()

#### Parsing XML with lxml.objectify

XML is another common structured data format supporting hierarchical, nested data with metadata. The book you are currently reading was actually created from a series of large XML documents.

Earlier, I showed the `pandas.read_html` function, which uses either lxml or Beautiful Soup under the hood to parse data from HTML. XML and HTML are structurally similar, but XML is more general. Here, I will show an example of how to use lxml to parse data from a more general XML format.

For many years, the New York Metropolitan Transportation Authority (MTA) published a number of data series about its bus and train services in XML format. Here we’ll look at the performance data, which is contained in a set of XML files. Each train or bus service has a different file (like *Performance_MNR.xml* for the Metro-North Railroad) containing monthly data as a series of XML records that look like this:

Using `lxml.objectify`, we parse the file and get a reference to the root node of the XML file with `getroot`:

In [ ]:
from lxml import objectify

path = "/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/mta_perf/Performance_MNR.xml"

with open(path) as f:
    parsed = objectify.parse(f)

root = parsed.getroot()

`root.INDICATOR `returns a generator yielding each `<INDICATOR>` XML element. For each record, we can populate a dictionary of tag names (like `YTD_ACTUAL`) to data values (excluding a few tags) by running the following code:



In [ ]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ", "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)
      

Lastly, convert this list of dictionaries into a DataFrame:



In [ ]:
perf = pd.DataFrame(data)

perf.head()

pandas's `pandas.read_xml` function turns this process into a one-line expression:



In [ ]:
perf2 = pd.read_xml(path)

perf2.head()

For more complex XML documents, refer to the docstring for `pandas.read_xml` which describes how to do selections and filters to extract a particular table of interest.


## 6.2 Binary Data Formats

One simple way to store (or *serialize*) data in binary format is using Python’s built-in `pickle` module. pandas objects all have a `to_pickle` method that writes the data to disk in pickle format:



In [ ]:
frame = pd.read_csv("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex1.csv")

frame

In [ ]:
frame.to_pickle("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/frame_pickle")

Pickle files are in general readable only in Python. You can read any "pickled" object stored in a file by using the built-in `pickle` directly, or even more conveniently using `pandas.read_pickle`:



In [ ]:
pd.read_pickle("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/frame_pickle")

pandas has built-in support for several other open source binary data formats, such as HDF5, ORC, and Apache Parquet. For example, if you install the `pyarrow` package (`conda install pyarrow`), then you can read Parquet files with `pandas.read_parquet`:



### Reading Microsoft Excel Files

pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the `pandas.ExcelFile` class or `pandas.read_excel` function. Internally, these tools use the add-on packages `xlrd` and `openpyxl` to read old-style XLS and newer XLSX files, respectively. These must be installed separately from pandas using pip or conda:

`conda install openpyxl xlrd`

To use `pandas.ExcelFile`, create an instance by passing a path to an `xls` or `xlsx` file:

In [ ]:
pip install xlrd openpyxl

In [ ]:
xlsx = pd.ExcelFile("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex1.xlsx")

In [ ]:
xlsx.sheet_names

Data stored in a sheet can then be read into DataFrame with `parse`:



In [ ]:
xlsx.parse(sheet_name="Sheet1")

This Excel table has an index column, so we can indicate that with the `index_col` argument:



In [ ]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

If you are reading multiple sheets in a file, then it is faster to create the `pandas.ExcelFile`, but you can also simply pass the filename to `pandas.read_excel`:



In [ ]:
frame = pd.read_excel("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex1.xlsx", sheet_name="Sheet1")

frame

To write pandas data to Excel format, you must first create an `ExcelWriter`, then write data to it using the pandas object's `to_excel` method:



In [ ]:
writer = pd.ExcelWriter("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex2.xlsx")

frame.to_excel(writer, "Sheet1")

In [ ]:
writer.close()

You can also pass a file path to `to_excel` and avoid the `ExcelWriter`:



In [ ]:
frame.to_excel("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/ex2.xlsx")

### Using HDF5 Format

HDF5 is a respected file format intended for storing large quantities of scientific array data. It is available as a C library, and it has interfaces available in many other languages, including Java, Julia, MATLAB, and Python. The “HDF” in HDF5 stands for *hierarchical data format*. Each HDF5 file can store multiple datasets and supporting metadata. Compared with simpler formats, HDF5 supports on-the-fly compression with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently. HDF5 can be a good choice for working with datasets that don't fit into memory, as you can efficiently read and write small sections of much larger arrays.

To get started with HDF5 and pandas, you must first install PyTables by installing the `tables` package with conda:

In [53]:
!brew install hdf5

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Downloading https://ghcr.io/v2/homebrew/portable-ruby/portable-ruby/blobs/sha256:ce5c135bf98da783bf5964e89aab587e3341c8adc379f36ace17ae7881f53e8c
######################################################################### 100.0%                                                  5.6%
==> Pouring portable-ruby-3.3.4_1.arm64_big_sur.bottle.tar.gz
==> Auto-updated Homebrew!
Updated 4 taps (heroku/brew, homebrew/services, homebrew/core and homebrew/cask).
==> New Formulae
aider               kubehound           mysql@8.4           serpl
clangql             libassuan@2         onion-location      sq
epoll-shim          libgedit-gfls       oxker               tabiew
ghc@9.8             libxpresent         packetry            usage
ipsw                mysql-client@8.4    progressline        uvw
==> New 

In [54]:
pip install tables

  Using cached tables-3.9.2.tar.gz (4.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numexpr-2.10.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (1.2 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached blosc2-2.7.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached ndindex-1.8-py3-none-any.whl.metadata (3.4 kB)
  Using cached msgpack-1.0.8-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.1 kB)
Using cached blosc2-2.7.1-cp312-cp312-macosx_11_0_arm64.whl (3.3 MB)
Using cached numexpr-2.10.1-cp312-cp312-macosx_11_0_arm64.whl (131 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
Using cached ndindex-1.8-py3-none-any.whl (91 kB)
Using cached msgpack-1.0.8-cp312-cp312-macosx_11_0_arm64.whl (85 kB)
  Created wheel for tables: filename=tables-3.9.2-cp312-cp312-macosx_11_0_arm64.whl size=3546076 sha256=64ec83df35d9b58007a87fe026d3

While it's possible to directly access HDF5 files using either the PyTables or h5py libraries, pandas provides a high-level interface that simplifies storing Series and DataFrame objects. The `HDFStore` class works like a dictionary and handles the low-level details:



In [55]:
import numpy as np

In [56]:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})

store = pd.HDFStore("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/mydata.h5")

store["obj1"] = frame

store["obj1_col"] = frame["a"]

store

<class 'pandas.io.pytables.HDFStore'>
File path: /Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/mydata.h5

Objects contained in the HDF5 file can then be retrieved with the same dictionary-like API:



In [58]:
store["obj1"]

,a
0,0.467645
1,0.528113
2,0.529589
3,0.889213
4,0.138362
...,...
95,0.808991
96,-0.619066
97,0.900500
98,0.226025


`HDFStore` supports two storage schemas, `"fixed"` and `"table"` (the default is `"fixed"`). The latter is generally slower, but it supports query operations using a special syntax:



In [62]:
store.put("obj2", frame, format="table")

store.select("obj2", where=["index >= 10 and index <=15"])

,a
10,-0.913811
11,1.399836
12,-0.576733
13,-0.270415
14,1.137300
15,-0.392134


In [63]:
store.close()

The `put` is an explicit version of the `store["obj2"] = frame` method but allows us to set other options like the storage format.

The `pandas.read_hdf` function gives you a shortcut to these tools:

In [65]:
frame.to_hdf("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/mydata.h5", "obj3", format="table")

/var/folders/fn/3w5085ns7nv4v62cfvj7m9sm0000gp/T/ipykernel_75892/3265367285.py:1: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  frame.to_hdf("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/mydata.h5", "obj3", format="table")


In [68]:
pd.read_hdf("/Users/timl/PythonWork/Data:Jupyter/McKinneyBook/Chapter678/mydata.h5", "obj3", where=["index < 5"])

,a
0,0.467645
1,0.528113
2,0.529589
3,0.889213
4,0.138362


If you'd like, you can delete the HDF5 file you created, like so:



## 6.3 Interacting with Web APIs

Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one method that I recommend is the requests package, which can be installed with pip or conda:

conda install requests
To find the last 30 GitHub issues for pandas on GitHub, we can make a `GET` HTTP request using the add-on `requests` library:

In [71]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [73]:
import requests

url = "https://api.github.com/repos/pandas-dev/pandas/issues"

resp = requests.get(url)

resp.raise_for_status()

resp

<Response [200]>

It's a good practice to always call `raise_for_status` after using `requests.get` to check for HTTP errors.

The response object’s `json` method will return a Python object containing the parsed JSON data as a dictionary or list (depending on what JSON is returned):

In [74]:
In [131]: data = resp.json()

In [132]: data[0]["title"]


'BUG Fix for Add numeric_only to function signature of DataFrameGroupBy.cumprod and `DataFrameGroupBy.cumsum '

Since the results retrieved are based on real-time data, what you see when you run this code will almost definitely be different.

Each element in `data` is a dictionary containing all of the data found on a GitHub issue page (except for the comments). We can pass `data` directly to `pandas.DataFrame` and extract fields of interest:

In [75]:
issues = pd.DataFrame(data, columns=["number", "title", 
                                     "labels", "state"])
issues

,number,title,labels,state
0,59427,BUG Fix for Add numeric_only to function signa...,"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
1,59422,BUG: UnboundLocalError when full outer merging...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
2,59421,BUG: merging DataFrames on a column containing...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,59418,BUG: Series.gt (and other comparison methods) ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,59417,Unable to build pandas from source on Windows,"[{'id': 129350, 'node_id': 'MDU6TGFiZWwxMjkzNT...",open
...,...,...,...,...
25,59380,BUG: Can't store `IntEnum` members inside a `p...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,59378,BUG: Inconsistent bar and line charts,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
27,59372,BUG: Python 3.13 development wheels not availa...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
28,59371,BUG: PeriodIndex.to_datetime inconsistent with...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


## 6.4 Interacting with Databases

In a business setting, a lot of data may not be stored in text or Excel files. SQL-based relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use, and many alternative databases have become quite popular. The choice of database is usually dependent on the performance, data integrity, and scalability needs of an application.

pandas has some functions to simplify loading the results of a SQL query into a DataFrame. As an example, I’ll create a SQLite3 database using Python’s built-in `sqlite3` driver:

In [76]:
import sqlite3

In [79]:
query = """
    CREATE TABLE test
    (a VARCHAR(20), b VARCHAR(20),
    c REAL, d INTEGER
    );"""

con = sqlite3.connect("mydata.sqlite")

con.execute(query)

In [80]:
con.commit()

Then, insert a few rows of data:



In [82]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)

In [83]:
con.commit()

Most Python SQL drivers return a list of tuples when selecting data from a table:



In [84]:
cursor = con.execute("SELECT * FROM test")

rows = cursor.fetchall()

rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

You can pass the list of tuples to the DataFrame constructor, but you also need the column names, contained in the cursor’s `description` attribute. Note that for SQLite3, the cursor `description` only provides column names (the other fields, which are part of Python's Database API specification, are `None`), but for some other database drivers, more column information is provided:



In [85]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [86]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


This is quite a bit of munging that you’d rather not repeat each time you query the database. The SQLAlchemy project is a popular Python SQL toolkit that abstracts away many of the common differences between SQL databases. pandas has a `read_sql` function that enables you to read data easily from a general SQLAlchemy connection. You can install SQLAlchemy with conda like so:

`pip install sqlalchemy`

Now, we'll connect to the same SQLite database with SQLAlchemy and read data from the table created before:

In [87]:
pip install sqlalchemy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [88]:
import sqlalchemy as sqla

db = sqla.create_engine("sqlite:///mydata.sqlite")

pd.read_sql("SELECT * FROM test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


Getting access to data is frequently the first step in the data analysis process. We have looked at a number of useful tools in this chapter that should help you get started. In the upcoming chapters we will dig deeper into data wrangling, data visualization, time series analysis, and other topics.

